In [14]:
!pip install vecstack

In [15]:
#Import all necessary packages

from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter 

import warnings
warnings.filterwarnings("ignore")

In [16]:
# Upload datasets

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

TrainingFile = r'/gdrive/My Drive/Projects/HomesiteKaggleProblem/RevisedHomesiteTrain.csv'
Training_Data = pd.read_csv(TrainingFile)

TestFile = r'/gdrive/My Drive/Projects/HomesiteKaggleProblem/RevisedHomesiteTest.csv'
Test_Data = pd.read_csv(TestFile)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [17]:
#Quick review of Training and Test Data info

print("Training Data Info")
Training_Data.info()
print()
print("Test Data Info")
Test_Data.info()

Training Data Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65000 entries, 0 to 64999
Columns: 596 entries, CoverageField11A to QuoteConversion_Flag
dtypes: float64(5), int64(591)
memory usage: 295.6 MB

Test Data Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173836 entries, 0 to 173835
Columns: 596 entries, CoverageField11A to GeographicField64
dtypes: float64(5), int64(590), object(1)
memory usage: 790.5+ MB


In [18]:
#Evaluating how balanced our training dataset is 

TotalQuotes=Training_Data["QuoteConversion_Flag"].value_counts()
nonconverted_quotes=np.round(100*TotalQuotes[0]/(TotalQuotes[0]+TotalQuotes[1]),2)
converted_quotes=np.round(100*TotalQuotes[1]/(TotalQuotes[0]+TotalQuotes[1]),2)

print('percentage of non-converted quotes', nonconverted_quotes, '%')
print('percentage of converted quotes', converted_quotes, '%')

percentage of non-converted quotes 81.14 %
percentage of converted quotes 18.86 %


In [19]:
#Deleting categorical column that is not in training data

del Test_Data['GeographicField64']
Test_Data.head()

#Adding column to Test Dataframe (YTest) for our predictions
Test_Data['QuoteConversion_Flag'] = ""

Test_Data.head()

,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,Field10,Field11,Field7,Field8,Field9,GeographicField10A,GeographicField10B,GeographicField11A,GeographicField11B,GeographicField12A,GeographicField12B,GeographicField13A,GeographicField13B,GeographicField14A,GeographicField14B,GeographicField15A,GeographicField15B,GeographicField16A,GeographicField16B,GeographicField17A,GeographicField17B,GeographicField18A,GeographicField18B,GeographicField19A,GeographicField19B,GeographicField1A,...,PropertyField7_R,PropertyField7_S,PropertyField7_T,PropertyField14_A,PropertyField14_B,PropertyField14_C,PropertyField14_D,PropertyField28_A,PropertyField28_B,PropertyField28_C,PropertyField28_D,PropertyField30_N,PropertyField30_Y,PropertyField31_K,PropertyField31_M,PropertyField31_N,PropertyField31_O,PropertyField32_N,PropertyField32_Y,PropertyField33_E,PropertyField33_F,PropertyField33_G,PropertyField33_H,PropertyField34_N,PropertyField34_Y,PropertyField36_N,PropertyField36_Y,PropertyField37_,PropertyField37_N,PropertyField37_Y,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX,QuoteConversion_Flag
0,13,22,4,4,4,4,3,3,3,4,13,22,13,23,1487.0,1.3045,16,0.9364,0.0006,-1,25,9,18,13,18,9,18,-1,19,10,13,8,17,2,10,-1,20,19,18,25,...,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,
1,4,5,8,14,8,14,7,12,8,13,13,22,13,23,564.0,1.1886,11,0.9919,0.0038,-1,25,4,15,8,13,5,14,-1,13,9,11,5,12,2,16,-1,22,23,24,17,...,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,
2,3,3,11,18,11,18,10,16,10,18,13,22,13,23,564.0,1.0670,15,0.8945,0.0038,-1,25,4,10,10,16,5,10,-1,11,11,18,5,13,2,13,-1,15,21,20,3,...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,1,0,0,1,0,0,0,0,0,
3,5,9,14,22,15,22,13,20,22,25,13,22,13,23,1113.0,1.2665,21,0.8870,0.0004,-1,25,24,24,23,23,24,24,25,25,23,22,24,25,2,7,-1,13,11,14,9,...,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,
4,12,21,4,5,4,5,4,4,4,5,13,22,13,23,935.0,1.0200,25,0.9153,0.0007,-1,25,2,3,3,5,2,5,-1,7,6,6,2,6,15,23,-1,17,4,8,2,...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,


In [20]:
#Compare training data and make sure it matches up with test data

Training_Data.head()

,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,Field10,Field11,Field7,Field8,Field9,GeographicField10A,GeographicField10B,GeographicField11A,GeographicField11B,GeographicField12A,GeographicField12B,GeographicField13A,GeographicField13B,GeographicField14A,GeographicField14B,GeographicField15A,GeographicField15B,GeographicField16A,GeographicField16B,GeographicField17A,GeographicField17B,GeographicField18A,GeographicField18B,GeographicField19A,GeographicField19B,GeographicField1A,...,PropertyField7_R,PropertyField7_S,PropertyField7_T,PropertyField14_A,PropertyField14_B,PropertyField14_C,PropertyField14_D,PropertyField28_A,PropertyField28_B,PropertyField28_C,PropertyField28_D,PropertyField30_N,PropertyField30_Y,PropertyField31_K,PropertyField31_M,PropertyField31_N,PropertyField31_O,PropertyField32_N,PropertyField32_Y,PropertyField33_E,PropertyField33_F,PropertyField33_G,PropertyField33_H,PropertyField34_N,PropertyField34_Y,PropertyField36_N,PropertyField36_Y,PropertyField37_,PropertyField37_N,PropertyField37_Y,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX,QuoteConversion_Flag
0,2,1,17,23,17,23,15,22,16,22,13,22,13,23,965.0,1.0200,23,0.9403,0.0006,-1,25,1,1,1,1,1,1,-1,7,1,2,1,2,13,22,-1,12,3,6,3,...,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,1,0,0,1,0,1,0,0,0,0
1,5,9,6,8,6,8,5,7,5,8,13,22,13,23,548.0,1.2433,7,1.0006,0.0040,-1,25,4,15,8,13,5,14,-1,13,9,11,5,12,2,16,-1,22,23,24,8,...,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0
2,4,6,7,12,7,12,6,10,7,11,25,25,13,23,548.0,1.2433,7,1.0006,0.0040,-1,25,4,11,9,15,5,12,-1,12,10,16,6,14,2,13,-1,17,21,21,2,...,1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0
3,15,23,3,2,3,2,2,2,3,2,13,22,13,23,1165.0,1.2665,10,0.9769,0.0004,-1,25,15,22,21,22,15,22,-1,15,24,23,9,18,2,3,-1,6,10,13,2,...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0
4,4,6,8,13,8,13,7,11,7,13,13,22,13,23,1487.0,1.3045,23,0.9472,0.0006,-1,25,9,17,12,16,9,17,-1,22,9,8,10,20,2,6,-1,4,17,16,2,...,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0


In [21]:
#Separate Train data and test data
XTrain = Training_Data[Training_Data.columns.difference(['QuoteConversion_Flag'])]
XTest = Test_Data[Test_Data.columns.difference(['QuoteConversion_Flag'])]
YTrain = Training_Data["QuoteConversion_Flag"]
#YTest = Test_Data["QuoteConversion_Flag"]

X_Train, X_Test, Y_Train, Y_Test = train_test_split(XTrain, YTrain, test_size = .30, random_state = 1)

In [22]:
#Fit and score decision tree model with no hyperparameter tuning
dt = DecisionTreeClassifier()
dt.fit(X_Train, Y_Train)
dt_predict=dt.predict(X_Test)
print()

print("Decision Tree Accuracy:{0:6f}".format(dt.score(X_Test,Y_Test)))  
print("Confusion Matrix - Decision Tree")
print(confusion_matrix(Y_Test,dt_predict))


Decision Tree Accuracy:0.881128
Confusion Matrix - Decision Tree
[[14702  1231]
 [ 1087  2480]]


In [23]:
#Print decision tree predictions to kaggle submission for comparison

dt_predict=dt.predict(XTest)
dt_prediction = pd.DataFrame({"QuoteNumber":XTest['QuoteNumber'],"QuoteConversion_Flag":dt_predict})  
print(dt_prediction['QuoteConversion_Flag'].value_counts())
dt_prediction.to_csv('/gdrive/My Drive/Projects/HomesiteKaggleProblem/DecisionTree_Submission.csv', index=False)

0    133125
1     40711
Name: QuoteConversion_Flag, dtype: int64


In [24]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestClassifier()
rfc.fit(X_Train, Y_Train)
rfc_predict=rfc.predict(X_Test)
print("accuracy Score (training) for RandomForest:{0:6f}".format(rfc.score(X_Test,Y_Test)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(Y_Test,rfc_predict))

accuracy Score (training) for RandomForest:0.908615
Confusion Matrix for Random Forest:
[[15696   237]
 [ 1545  2022]]


In [25]:
#Print random forest predictions to kaggle submission for comparison

rfc_predict=rfc.predict(XTest)
rf_prediction = pd.DataFrame({"QuoteNumber":XTest['QuoteNumber'],"QuoteConversion_Flag":rfc_predict})  
print(rf_prediction['QuoteConversion_Flag'].value_counts())
rf_prediction.to_csv('/gdrive/My Drive/Projects/HomesiteKaggleProblem/RandomForest_Submission.csv', index=False)

0    152657
1     21179
Name: QuoteConversion_Flag, dtype: int64


In [26]:
#Gradient Boosting ============================================================
search_grid={'n_estimators':[5,10,20,30,50],'learning_rate':[0.01,.1]}
gb =GradientBoostingClassifier()
gb.fit(X_Train, Y_Train)
gb_predict=gb.predict(X_Test)
print("accuracy Score (training) for Boosting:{0:6f}".format(gb.score(X_Test,Y_Test)))
print("Confusion Matrix for boosting:")
print(confusion_matrix(Y_Test,gb_predict))

accuracy Score (training) for Boosting:0.923436
Confusion Matrix for boosting:
[[15737   196]
 [ 1297  2270]]


In [27]:
#Print gradient boosting predictions to kaggle submission for comparison

gb_predict=gb.predict(XTest)
gb_prediction = pd.DataFrame({"QuoteNumber":XTest['QuoteNumber'],"QuoteConversion_Flag":gb_predict})  
print(gb_prediction['QuoteConversion_Flag'].value_counts())
gb_prediction.to_csv('/gdrive/My Drive/Projects/HomesiteKaggleProblem/GradientBoosting_Submission.csv', index=False)

0    150704
1     23132
Name: QuoteConversion_Flag, dtype: int64


In [28]:
#Construct Multilayer Perceptron using the best parameters
#Default mode
mlp = MLPClassifier()
mlp.fit(X_Train, Y_Train)
mlp_predict=mlp.predict(X_Test)
print("accuracy Score (training) for Multilayer Perceptron:{0:6f}".format(mlp.score(X_Test,Y_Test)))
print("Confusion Matrix for Multilayer Perceptron:")
print(confusion_matrix(Y_Test,mlp_predict))

accuracy Score (training) for Multilayer Perceptron:0.416051
Confusion Matrix for Multilayer Perceptron:
[[ 4637 11296]
 [   91  3476]]


In [29]:
#Print Multi-Layer Perceptron predictions to kaggle submission for comparison

mlp_predict=mlp.predict(XTest)
mlp_prediction = pd.DataFrame({"QuoteNumber":XTest['QuoteNumber'],"QuoteConversion_Flag":mlp_predict})  
print(mlp_prediction['QuoteConversion_Flag'].value_counts())
dt_prediction.to_csv('/gdrive/My Drive/Projects/HomesiteKaggleProblem/MultiLayerPerceptron_Submission.csv', index=False)

1    162727
0     11109
Name: QuoteConversion_Flag, dtype: int64


In [30]:
#Construct Support Vector Machines using the best parameters
#Default mode
svc = svm.SVC()
svc.fit(X_Train, Y_Train)
svc_predict=svc.predict(X_Test)
print("accuracy Score (training) for Support Vector Machines:{0:6f}".format(svc.score(X_Test,Y_Test)))
print("Confusion Matrix for Support Vector Machines:")
print(confusion_matrix(Y_Test,svc_predict))

accuracy Score (training) for Support Vector Machines:0.817077
Confusion Matrix for Support Vector Machines:
[[15933     0]
 [ 3567     0]]


In [31]:
#Construct KNN
knn = KNeighborsClassifier()
knn.fit(X_Train, Y_Train)
knn_predict = knn.predict(X_Test)
print("accuracy Score (training) for K Nearest Neighbors:{0:6f}".format(knn.score(X_Test,Y_Test)))
print("Confusion Matrix for K Nearest Neighbors:")
print(confusion_matrix(Y_Test,knn_predict))

accuracy Score (training) for K Nearest Neighbors:0.791179
Confusion Matrix for K Nearest Neighbors:
[[15190   743]
 [ 3329   238]]


In [32]:
#Print k nearest neighbor predictions to kaggle submission for comparison

knn_predict=knn.predict(XTest)
knn_prediction = pd.DataFrame({"QuoteNumber":XTest['QuoteNumber'],"QuoteConversion_Flag":knn_predict})  
print(knn_prediction['QuoteConversion_Flag'].value_counts())
knn_prediction.to_csv('/gdrive/My Drive/Projects/HomesiteKaggleProblem/KNN_Submission.csv', index=False)

0    158275
1     15561
Name: QuoteConversion_Flag, dtype: int64


In [33]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(Y_Train))
sm = SMOTE(sampling_strategy='float', ratio=0.5)
X_res, y_res = sm.fit_resample(X_Train, Y_Train)
print('Resampled dataset shape %s' % Counter(y_res))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 36805, 1: 8695})
Resampled dataset shape Counter({0: 36805, 1: 18402})


In [34]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ GradientBoostingClassifier(), RandomForestClassifier(), DecisionTreeClassifier() ]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, X_Test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingClassifier]
    fold  0:  [0.93341545]
    fold  1:  [0.93529923]
    fold  2:  [0.93131430]
    fold  3:  [0.93543946]
    ----
    MEAN:     [0.93386711] + [0.00167661]
    FULL:     [0.93386708]

model  1:     [RandomForestClassifier]
    fold  0:  [0.92131575]
    fold  1:  [0.92276482]
    fold  2:  [0.92059122]
    fold  3:  [0.92486052]
    ----
    MEAN:     [0.92238308] + [0.00163044]
    FULL:     [0.92238303]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.89689900]
    fold  1:  [0.89697145]
    fold  2:  [0.89871033]
    fold  3:  [0.90123904]
    ----
    MEAN:     [0.89845496] + [0.00176339]
    FULL:     [0.89845491]



In [35]:
#STACKING - CONTRUCT A GRADIENT BOOSTING MODEL==============================
model = GradientBoostingClassifier()
    
model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(Y_Test, y_pred))
print("Confusion Matrix after STACKING for Boosting:")
print(confusion_matrix(Y_Test,y_pred))
print("=== Classification Report ===")
print(classification_report(Y_Test,y_pred))

Final prediction score for ensemble methods: [0.92230769]
Confusion Matrix after STACKING for Boosting:
[[15590   343]
 [ 1172  2395]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.93      0.98      0.95     15933
           1       0.87      0.67      0.76      3567

    accuracy                           0.92     19500
   macro avg       0.90      0.82      0.86     19500
weighted avg       0.92      0.92      0.92     19500



In [36]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ GradientBoostingClassifier(), RandomForestClassifier(), DecisionTreeClassifier() ]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, XTest,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingClassifier]
    fold  0:  [0.93341545]
    fold  1:  [0.93529923]
    fold  2:  [0.93131430]
    fold  3:  [0.93543946]
    ----
    MEAN:     [0.93386711] + [0.00167661]
    FULL:     [0.93386708]

model  1:     [RandomForestClassifier]
    fold  0:  [0.92124330]
    fold  1:  [0.92182292]
    fold  2:  [0.92312708]
    fold  3:  [0.92493298]
    ----
    MEAN:     [0.92278157] + [0.00141715]
    FULL:     [0.92278153]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.89682655]
    fold  1:  [0.90030430]
    fold  2:  [0.90226054]
    fold  3:  [0.90109412]
    ----
    MEAN:     [0.90012138] + [0.00202556]
    FULL:     [0.90012136]



In [37]:
#HYPERPARAMETER TUNING FOR GB STACKED MODEL
#Gradient Boosting ============================================================
search_grid={'n_estimators':[5,10,20, 30, 50],'learning_rate':[0.01,.1]}
abc =GradientBoostingClassifier()
abc.fit(X_Train, Y_Train)
abc_predict=abc.predict(X_Test)
print("accuracy Score (training) for Boosting:{0:6f}".format(abc.score(X_Test,Y_Test)))
print("Confusion Matrix for boosting:")
print(confusion_matrix(Y_Test,abc_predict))

abc_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
abc_random.fit(X_Train, Y_Train)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)

model = GradientBoostingClassifier(**grid_parm_abc)
model.fit(S_Train, y_res)
Test_Prediction = model.predict(S_Test)

#Write gb stacked predicted model to kaggle submission file
Test_Prediction = model.predict(S_Test)
Prediction = pd.DataFrame({"QuoteNumber":XTest['QuoteNumber'],"QuoteConversion_Flag":Test_Prediction})  

print(Prediction['QuoteConversion_Flag'].value_counts())
Prediction.to_csv('/gdrive/My Drive/Projects/HomesiteKaggleProblem/StackedGradientBoosting_Submission.csv', index=False)

accuracy Score (training) for Boosting:0.923436
Confusion Matrix for boosting:
[[15737   196]
 [ 1297  2270]]
{'n_estimators': 50, 'learning_rate': 0.1}
0    148719
1     25117
Name: QuoteConversion_Flag, dtype: int64


In [38]:
#Get Prediction Probability for the predicted class as a dataframe
pred_Probability =pd.DataFrame(model.predict_proba(S_Test))

pred_Probability.head()

,0,1
0,0.947864,0.052136
1,0.947864,0.052136
2,0.947864,0.052136
3,0.947864,0.052136
4,0.947864,0.052136
